In [31]:
import numpy as np

In [1]:
import pandas as pd
import time
import os
import pickle


good_data = pd.read_csv("./dataset/good.csv")
bad_data = pd.read_csv("./dataset/bad.csv")

## 데이터 전처리

In [183]:
good_data = good_data.astype(int)
bad_data = bad_data.astype(int)

if os.path.exists(f'./dataset/fix_good_datas') and os.path.exists(f'./dataset/fix_bad_data') and os.path.exists(f'./dataset/drop_cols'):
    fix_good_data = pickle.load(open(f'./fix_good_data','rb'))
    fix_bad_data = pickle.load(open(f'./fix_bad_data','rb'))
    drop_cols = pickle.load(open(f'./drop_cols','rb'))
    
else:

    drop_cols = ['Unnamed: 0', '.', '+', '-',  '/', ',', '및', '은', '을', '의', '첫', '하기', '를', '3', '된', '매', 
                 '10', '5', '에', '에서', '완료', '!', '과', '있습니다', '만', '1', '8', '50', '는', '가', '고', '가장',
                  '=', '2', '중','한', '하고', '4', '위', ')', '(', '100','30', '?', '하여', 'ㄴ', '것', '입니다', '많은',
                  '합니다', '6', '시', '모든', '다', '와', '곳', '하는', '도', '적', '인', '할', '즈', '_', '경우',
                  '우리', '일', '더', '9', '7', '0', '수', '들', ':', ']', '[', '으로', '없는', '&', '등', '로', '있는',
                   '이', '충', '|', '\'', '\'', '\"', '경기', '저희', '호텔', '365']
    for col in bad_data.columns:

        vacuum_check = col.strip()

        if bad_data[col].sum() < 5000 or len(vacuum_check) == 0:
            drop_cols.append(col)

    fix_good_data = good_data.drop(['Unnamed: 0.1'], axis=1)
    fix_good_data = fix_good_data.drop(drop_cols, 1)
    fix_bad_data = bad_data.drop(drop_cols, 1)
    
    fix_good_data['label'] = 0
    fix_bad_data['label'] = 1

    pickle.dump(fix_good_data, open(f'./dataset/fix_good_data','wb'))
    pickle.dump(fix_bad_data, open(f'./dataset/fix_bad_data','wb'))
    pickle.dump(drop_cols, open(f'./dataset/drop_cols','wb'))

## 데이터셋 구축

In [190]:
# 데이터셋 나누기 학습용 90% 테스트용 10%
good_train = fix_good_data.sample(frac=0.9)
good_test = fix_good_data.drop(good_train.index)

bad_train = fix_bad_data.sample(frac=0.9)
bad_test = fix_bad_data.drop(bad_train.index)

train = pd.concat([good_train, bad_train], axis=0)
test = pd.concat([good_test, bad_test], axis=0)

# Shuffle
train = train.sample(frac=1)
test = test.sample(frac=1)

x_train = train.iloc[:, :-1].to_numpy()
y_train = train.iloc[:, -1:].to_numpy()

x_test = test.iloc[:, :-1].to_numpy()
y_test = test.iloc[:, -1:].to_numpy()

In [207]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from keras.callbacks import ModelCheckpoint

In [208]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

checkpoint_path = 'dataset/cp'

checkpoint = ModelCheckpoint(checkpoint_path, 
                            save_best_only=True, 
                            save_weights_only=True, 
                            monitor='val_loss',
                            verbose=1)


model = Sequential([

Dense(128, activation='relu' ,input_shape=[14]),
Dense(64, activation='relu'),
Dense(32, activation='relu'),
Dense(16, activation='relu'),
Dense(2, activation='softmax')
])

model.compile(optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])

model.fit(x_train, y_train, shuffle=True, epochs=100, verbose=1, callbacks=[checkpoint],)

model.evaluate(x_test, y_test)

Epoch 1/100
28/28 [==============================] - 0s 3ms/step - loss: 0.4015 - accuracy: 0.8364
Epoch 2/100
28/28 [==============================] - 0s 4ms/step - loss: 0.2890 - accuracy: 0.9268
Epoch 3/100
28/28 [==============================] - 0s 4ms/step - loss: 0.2294 - accuracy: 0.9371
Epoch 4/100
28/28 [==============================] - 0s 4ms/step - loss: 0.2180 - accuracy: 0.9359
Epoch 5/100
28/28 [==============================] - 0s 4ms/step - loss: 0.1885 - accuracy: 0.9405
Epoch 6/100
28/28 [==============================] - 0s 4ms/step - loss: 0.1441 - accuracy: 0.9485
Epoch 7/100
28/28 [==============================] - 0s 4ms/step - loss: 0.1584 - accuracy: 0.9462
Epoch 8/100
28/28 [==============================] - 0s 4ms/step - loss: 0.1412 - accuracy: 0.9508
Epoch 9/100
28/28 [==============================] - 0s 4ms/step - loss: 0.1780 - accuracy: 0.9405
Epoch 10/100
28/28 [==============================] - 0s 4ms/step - loss: 0.1429 - accuracy: 0.9462
Epoch 11/

Epoch 47/100
28/28 [==============================] - 0s 4ms/step - loss: 0.0959 - accuracy: 0.9634
Epoch 48/100
28/28 [==============================] - 0s 4ms/step - loss: 0.0934 - accuracy: 0.9622
Epoch 49/100
28/28 [==============================] - 0s 4ms/step - loss: 0.0938 - accuracy: 0.9634
Epoch 50/100
28/28 [==============================] - 0s 4ms/step - loss: 0.0929 - accuracy: 0.9634
Epoch 51/100
28/28 [==============================] - 0s 4ms/step - loss: 0.0925 - accuracy: 0.9645
Epoch 52/100
28/28 [==============================] - 0s 4ms/step - loss: 0.0923 - accuracy: 0.9645
Epoch 53/100
28/28 [==============================] - 0s 4ms/step - loss: 0.0918 - accuracy: 0.9657
Epoch 54/100
28/28 [==============================] - 0s 4ms/step - loss: 0.1745 - accuracy: 0.9416
Epoch 55/100
28/28 [==============================] - 0s 4ms/step - loss: 0.1167 - accuracy: 0.9600
Epoch 56/100
28/28 [==============================] - 0s 4ms/step - loss: 0.1051 - accuracy: 0.9588


Epoch 93/100
28/28 [==============================] - 0s 4ms/step - loss: 0.0879 - accuracy: 0.9657
Epoch 94/100
28/28 [==============================] - 0s 4ms/step - loss: 0.0895 - accuracy: 0.9645
Epoch 95/100
28/28 [==============================] - 0s 4ms/step - loss: 0.0887 - accuracy: 0.9657
Epoch 96/100
28/28 [==============================] - 0s 4ms/step - loss: 0.0887 - accuracy: 0.9657
Epoch 97/100
28/28 [==============================] - 0s 4ms/step - loss: 0.0925 - accuracy: 0.9668
Epoch 98/100
28/28 [==============================] - 0s 4ms/step - loss: 0.0888 - accuracy: 0.9657
Epoch 99/100
28/28 [==============================] - 0s 4ms/step - loss: 0.0881 - accuracy: 0.9634
Epoch 100/100
4/4 [==============================] - 0s 2ms/step - loss: 0.3031 - accuracy: 0.9694


[0.3030923008918762, 0.9693877696990967]